# PITN - DWI Downsampling and DTI Fitting

Code by:

Tyler Spears - tas6hh@virginia.edu

Dr. Tom Fletcher

## Imports & Environment Setup

In [ ]:
# Automatically re-import project-specific modules.
%load_ext autoreload
%autoreload 1

# imports
import collections
import functools
import io
import math
import itertools
import os
import shutil
import pathlib
import inspect
import random
import subprocess
import sys
import warnings
from pathlib import Path
import re
import json

import ants
import dipy
import dipy.core
import dipy.reconst
import dipy.reconst.dti
import dipy.segment.mask
import dipy.viz
import dipy.viz.regtools
import dotenv

# visualization libraries
%matplotlib inline
import matplotlib as mpl
import mpl_toolkits
import matplotlib.pyplot as plt

import IPython

# Try importing GPUtil for printing GPU specs.
# May not be installed if using CPU only.
try:
    import GPUtil
except ImportError:
    warnings.warn("WARNING: Package GPUtil not found, cannot print GPU specs")

# Data management libraries.
import nibabel as nib
import natsort
from natsort import natsorted
import box
from box import Box
import pprint
from pprint import pprint as ppr

# Computation & ML libraries.
import numpy as np
import skimage
import torch
import torchio

plt.rcParams.update({"figure.autolayout": True})
plt.rcParams.update({"figure.facecolor": [1.0, 1.0, 1.0, 1.0]})

# Set print options for ndarrays/tensors.
np.set_printoptions(suppress=True, edgeitems=2, threshold=100, linewidth=88)
torch.set_printoptions(
    sci_mode=False, edgeitems=2, threshold=100, linewidth=88, profile="short"
)

In [ ]:
# Update notebook's environment variables with direnv.
# This requires the python-dotenv package, and direnv be installed on the system
# This will not work on Windows.
# NOTE: This is kind of hacky, and not necessarily safe. Be careful...
# Libraries needed on the python side:
# - os
# - subprocess
# - io
# - dotenv

# Form command to be run in direnv's context. This command will print out
# all environment variables defined in the subprocess/sub-shell.
command = f"direnv exec {os.getcwd()} /usr/bin/env"
# Run command in a new subprocess.
proc = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True, cwd=os.getcwd())
# Store and format the subprocess' output.
proc_out = proc.communicate()[0].strip().decode("utf-8")
# Use python-dotenv to load the environment variables by using the output of
# 'direnv exec ...' as a 'dummy' .env file.
dotenv.load_dotenv(stream=io.StringIO(proc_out), override=True);

In [ ]:
# Project-specific scripts
# It's easier to import it this way rather than make an entirely new package, due to
# conflicts with local packages and anaconda installations.
# You made me do this, poor python package management!!
if "PROJECT_ROOT" in os.environ:
    lib_location = str(Path(os.environ["PROJECT_ROOT"]).resolve())
else:
    lib_location = str(Path("../../").resolve())
if lib_location not in sys.path:
    sys.path.insert(0, lib_location)
import lib as pitn

# Include the top-level lib module along with its submodules.
%aimport lib
# Grab all submodules of lib, not including modules outside of the package.
includes = list(
    filter(
        lambda m: m.startswith("lib."),
        map(lambda x: x[1].__name__, inspect.getmembers(pitn, inspect.ismodule)),
    )
)
# Run aimport magic with constructed includes.
ipy = IPython.get_ipython()
ipy.run_line_magic("aimport", ", ".join(includes))

In [ ]:
# torch setup
# allow for CUDA usage, if available
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
# keep device as the cpu
# device = torch.device('cpu')
print(device)

### Specs Recording

In [ ]:
%%capture --no-stderr cap
# Capture output and save to log. Needs to be at the *very first* line of the cell.
# Watermark
%load_ext watermark
%watermark --author "Tyler Spears" --updated --iso8601  --python --machine --iversions --githash
if torch.cuda.is_available():

    # GPU information
    # Taken from
    # <https://www.thepythoncode.com/article/get-hardware-system-information-python>.
    # If GPUtil is not installed, skip this step.
    try:
        gpus = GPUtil.getGPUs()
        print("=" * 50, "GPU Specs", "=" * 50)
        list_gpus = []
        for gpu in gpus:
            # get the GPU id
            gpu_id = gpu.id
            # name of GPU
            gpu_name = gpu.name
            driver_version = gpu.driver
            cuda_version = torch.version.cuda
            # get total memory
            gpu_total_memory = f"{gpu.memoryTotal}MB"
            gpu_uuid = gpu.uuid
            list_gpus.append(
                (
                    gpu_id,
                    gpu_name,
                    driver_version,
                    cuda_version,
                    gpu_total_memory,
                    gpu_uuid,
                )
            )

        print(
            tabulate(
                list_gpus,
                headers=(
                    "id",
                    "Name",
                    "Driver Version",
                    "CUDA Version",
                    "Total Memory",
                    "uuid",
                ),
            )
        )
    except NameError:
        print("CUDA Version: ", torch.version.cuda)

else:
    print("CUDA not in use, falling back to CPU")

In [ ]:
# cap is defined in an ipython magic command
print(cap)

Author: Tyler Spears

Last updated: 2021-10-10T22:24:19.957916+00:00

Python implementation: CPython
Python version       : 3.8.8
IPython version      : 7.23.1

Compiler    : GCC 7.3.0
OS          : Linux
Release     : 5.4.0-88-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 8
Architecture: 64bit

Git hash: a0881885f8686eed0a7e92bc43c4517870ae4a19

sys       : 3.8.8 (default, Feb 24 2021, 21:46:12) 
[GCC 7.3.0]
numpy     : 1.20.2
dipy      : 1.4.1
GPUtil    : 1.4.0
natsort   : 7.1.1
nibabel   : 3.2.1
monai     : 0.7.dev2138
IPython   : 7.23.1
skimage   : 0.18.1
box       : 5.4.1
ants      : 0.2.7
matplotlib: 3.4.1
json      : 2.0.9
torch     : 1.9.0
seaborn   : 0.11.1
torchio   : 0.18.37

================================================== GPU Specs ==================================================
  id  Name       Driver Version      CUDA Version  Total Memory    uuid
----  ---------  ----------------  --------------  --------------  -----------------------------------

### Data Variables & Definitions Setup

In [ ]:
# Set up directories
data_dir = pathlib.Path(os.environ["DATA_DIR"])
processed_data_dir = pathlib.Path(os.environ["WRITE_DATA_DIR"])
hcp_source_data_dir = data_dir / "hcp"
oasis_source_data_dir = data_dir / "oasis3"
assert hcp_source_data_dir.exists() and oasis_source_data_dir.exists()

hcp_processed_data_dir = processed_data_dir / "hcp/derivatives/mean-downsample"
oasis_processed_data_dir = processed_data_dir / "oasis3/derivatives/mean-downsample"
assert hcp_processed_data_dir.exists() and oasis_processed_data_dir.exists()

In [ ]:
# Dict to keep track of experiment configuration parameters. Will not be logged to
# tensorboard.
params = Box(default_box=True)
# 6 channels for the 6 DTI components
params.channels = 6

# Voxel sizes for the mean downsampling.
# This is really just the HCP space size.
params.source_vox_size = 1.25
# OASIS3 DWI spacing.
params.target_vox_size = 2.0
params.downsample_factor = params.target_vox_size / params.source_vox_size
# Include all b-values for DTI fitting.
params.bval_range = (0, 4000)
params.dti_fit_method = "WLS"

# Percentile range of DWIs that will have voxel intensities clamped.
# In other words, for each subject, for each DWI, any voxel
# values <= the first quantile value will be clamped to that quantile, and any voxels
# >= the second quantile value will be clamped to that quantile.
params.clamp_percentiles = (0.01, 99.99)

In [ ]:
# Set the output directory based on the target voxel size.
oasis_processed_data_dir /= "scale-orig"
assert oasis_processed_data_dir.exists()

## DTI Fitting

In [ ]:
# Define pipeline
# Set up the transformation pipeline.

# Initial import and selection of DWIs.
dwi_processing_transforms = torchio.Compose(
    [
        torchio.transforms.ToCanonical(include=("dwi", "mask"), copy=False),
        pitn.transforms.BValSelectionTransform(
            bval_range=params.bval_range,
            bval_key="bvals",
            bvec_key="bvecs",
            include="dwi",
            copy=False,
        ),
    ]
)

# Percentile clipping of values in DWIs to remove outliers assumed to be noise.
intensity_clip_transform = pitn.transforms.ClipPercentileTransformd(
    "dwi",
    lower=params.clamp_percentiles[0],
    upper=params.clamp_percentiles[1],
    only_nonzero=True,
    channel_wise=True,
)

# Final pipeline for downsampling the DWIs and fitting to DTIs.
downsample_dti_fitting_transforms = torchio.Compose(
    [
        pitn.transforms.FractionalMeanDownsampleTransform(
            source_vox_size=params.source_vox_size,
            target_vox_size=params.target_vox_size,
            include=("dwi", "mask"),
            keep={"dwi": "fr_dwi", "mask": "fr_mask"},
            copy=False,
        ),
        pitn.transforms.RenameImageTransform(
            {"dwi": "lr_dwi", "mask": "lr_mask"}, copy=False
        ),
        pitn.transforms.FitDTITransform(
            "bvals",
            "bvecs",
            "fr_mask",
            fit_method=params.dti_fit_method,
            include=("fr_dwi"),
            #             cache_dir="./.cache",
            copy=False,
        ),
        pitn.transforms.FitDTITransform(
            "bvals",
            "bvecs",
            "lr_mask",
            fit_method=params.dti_fit_method,
            include=("lr_dwi"),
            #             cache_dir="./.cache",
            copy=False,
        ),
        pitn.transforms.RenameImageTransform(
            {"fr_dwi": "fr_dti", "lr_dwi": "lr_dti"}, copy=False
        ),
        pitn.transforms.ImageToDictTransform(include=("lr_dti", "lr_mask"), copy=False),
    ]
)

### HCP - DWI Downsampling & DTI Fitting

In [ ]:
# Load and process DWIs, downsample DWIs, and fit DTIs to both full resolution and low-
# resolution DTIs.
hcp_subj_data: dict = dict()

# HCP has error-corrected DWIs and a matching mask. So, grab the DWI and the mask
# from the same source directory.
source_dwi_dir = "T1w/Diffusion"
source_dwi_filename = "data.nii.gz"
source_bval_filename = "bvals"
source_bvec_filename = "bvecs"
source_mask_dir = source_dwi_dir
mask_filename = "nodif_brain_mask.nii.gz"

fr_output_dir = hcp_processed_data_dir / "scale-orig"
lr_output_dir = hcp_processed_data_dir / f"scale-{params.target_vox_size:.2f}mm"

# HCP subj directories are only numbers, no 'sub-' or code in front.
for subj_dir in hcp_source_data_dir.glob("[0-9]*"):
    subj_id = subj_dir.name
    # Output filenames, stored in a Box for convenience.
    filenames = Box(default_box=True)
    filenames.fr.dti = f"sub-{subj_id}_scale-orig_dti.nii.gz"
    filenames.fr.mask = f"sub-{subj_id}_scale-orig_mask.nii.gz"
    filenames.lr.dti = f"sub-{subj_id}_scale-{params.target_vox_size:.2f}mm_dti.nii.gz"
    filenames.lr.mask = (
        f"sub-{subj_id}_scale-{params.target_vox_size:.2f}mm_mask.nii.gz"
    )
    # Set the subject-specific data source directory.
    subj_source_dir = subj_dir / source_dwi_dir

    bvals = torch.as_tensor(
        np.loadtxt(subj_source_dir / source_bval_filename).astype(int)
    )
    bvecs = torch.as_tensor(np.loadtxt(subj_source_dir / source_bvec_filename))
    # Reshape to be N x 3
    if bvecs.shape[0] == 3:
        bvecs = bvecs.T

    dwi = torchio.ScalarImage(
        subj_source_dir / source_dwi_filename,
        type=torchio.INTENSITY,
        bvals=bvals,
        bvecs=bvecs,
        reader=pitn.io.nifti_reader,
        channels_last=True,
    )

    subj_mask_dir = subj_dir / source_mask_dir
    brain_mask = torchio.LabelMap(
        subj_mask_dir / mask_filename,
        type=torchio.LABEL,
        channels_last=False,
    )
    brain_mask.set_data(brain_mask.data.bool())
    
    # Create the initial Subject object with the DWI and the corresponding mask.
    subj_dict = torchio.Subject(subj_id=subj_id, dwi=dwi, mask=brain_mask)
    
    # Perform some light processing on the DWI.
    dwi_preproc = dwi_processing_transforms(subj_dict)
    # Clip extreme values on a per-channel basis of the masked DWI.
    clipped_dwi_dict = intensity_clip_transform(
        {"dwi": dwi_preproc["dwi"].tensor * dwi_preproc["mask"].tensor}
    )
    dwi_preproc["dwi"].set_data(clipped_dwi_dict["dwi"])
    
    # Downsample the DWIs, and fit both the full-resolution and low-resolution DWIs to
    # DTIs.
    preproc_subj = downsample_dti_fitting_transforms(dwi_preproc)

    # Save out all images to files.
    header = nib.load(subj_source_dir / source_dwi_filename).header
    # FR images.
    subj_fr_output_dir = fr_output_dir / f"sub-{subj_id}/dti"
    subj_fr_output_dir.mkdir(parents=True, exist_ok=True)
    dti_img = nib.Nifti1Image(
        preproc_subj["fr_dti"]["data"].cpu().numpy(),
        affine=preproc_subj["fr_dti"]["affine"],
        header=header,
    )
    mask_img = nib.Nifti1Image(
        preproc_subj["fr_mask"]["data"].cpu().numpy(),
        affine=preproc_subj["fr_mask"]["affine"],
        header=header,
    )

    nib.save(dti_img, subj_fr_output_dir / filenames.fr.dti)
    nib.save(mask_img, subj_fr_output_dir / filenames.fr.mask)
    # Save some of the pipeline description
    fr_pipeline_dir = subj_fr_output_dir / "pipeline"
    fr_pipeline_dir.mkdir(parents=True, exist_ok=True)
    fr_pipe_file = fr_pipeline_dir / "description.txt"
    with open(fr_pipe_file, "a+") as f:
        f.write("Pipeline Description\n")
        f.write(f"Voxel spacing: {params.source_vox_size}\n")
        f.write(f"dipy version {dipy.__version__}\n")
        f.write(f"Params {params.to_dict()}\n")
        f.write(
            "Pipeline functions:\n"
            + f"{(dwi_processing_transforms, intensity_clip_transform, downsample_dti_fitting_transforms)}\n"
        )

    # LR images.
    subj_lr_output_dir = lr_output_dir / f"sub-{subj_id}/dti"
    subj_lr_output_dir.mkdir(parents=True, exist_ok=True)
    dti_img = nib.Nifti1Image(
        preproc_subj["lr_dti"]["data"].cpu().numpy(),
        affine=preproc_subj["lr_dti"]["affine"],
        header=header,
    )
    mask_img = nib.Nifti1Image(
        preproc_subj["lr_mask"]["data"].cpu().numpy(),
        affine=preproc_subj["lr_mask"]["affine"],
        header=header,
    )

    nib.save(dti_img, subj_lr_output_dir / filenames.lr.dti)
    nib.save(mask_img, subj_lr_output_dir / filenames.lr.mask)
    # Save some of the pipeline description"
    lr_pipeline_dir = subj_lr_output_dir / "pipeline"
    lr_pipeline_dir.mkdir(parents=True, exist_ok=True)
    lr_pipe_file = lr_pipeline_dir / "description.txt"
    with open(lr_pipe_file, "a+") as f:
        f.write("Pipeline Description\n")
        f.write(f"Original voxel spacing: {params.source_vox_size}\n")
        f.write(f"Downscaled voxel spacing: {params.target_vox_size}\n")
        f.write(f"dipy version {dipy.__version__}\n")
        f.write(f"Params {params.to_dict()}\n")
        f.write(
            "Pipeline functions:\n"
            + f"{(dwi_processing_transforms, intensity_clip_transform, downsample_dti_fitting_transforms)}\n"
        )

    # Optionally save into a dict for later processing.
    #     hcp_subj_data[subj_id] = preproc_subj

    print("=" * 20)

print("===Data Loaded & Transformed===")

### OASIS3 - DTI Fitting

In [ ]:
# Pipeline for (only) fitting DTIs, no downscaling.
dti_fitting_transforms = torchio.Compose(
    [
        pitn.transforms.FitDTITransform(
            "bvals",
            "bvecs",
            "mask",
            fit_method=params.dti_fit_method,
            include=("dwi"),
            #             cache_dir="./.cache",
            copy=False,
        ),
        pitn.transforms.RenameImageTransform({"dwi": "dti", "dwi": "dti"}, copy=False),
    ]
)

In [ ]:
# Load and process DWIs, downsample DWIs, and fit DTIs to both full resolution and low-
# resolution DTIs.
oasis_subj_data: dict = dict()

# OASIS source data files are stored in a BIDS-like structure with
# `sub-OAS3[subj_number]/ses-[session_id]/[image_type]`
for subj_dir in oasis_source_data_dir.glob("sub-OAS[0-9]*"):
    
    # Individual scans can be broken out by 1) subject id, 2) session id, and optionally
    # 3) run number.
    subj_id = subj_dir.name
    sessions = list(subj_dir.glob("ses-*"))
    sessions = natsorted(sessions)
    # Just take the first available session.
    session_id = sessions[0].name
    # Find the available run numbers, if any.
    runs = (subj_dir / session_id / "dwi").glob("*run-*")
    runs = set(map(lambda s: re.search(r"run-[0-9]+", s.name)[0], runs))
    runs = natsorted(list(runs))
    # If more than one run exists, pick the first one (usually "01"). Otherwise, set the
    # id to be an empty string for later.
    if len(runs) == 0:
        run_id = ""
    else:
        run_id = runs[0]
    # Construct the freesurfer-specific subject id, which is a combination of subject
    # id and session id.
    freesurfer_id = f'{subj_id.replace("sub-", "")}_MR_{session_id.replace("ses-", "")}'

    # The subject's DWI directory
    subj_dwi_dir = subj_dir / session_id / "dwi"
    read_files = Box(default_box=True)
    read_files.dwi = list(
        subj_dwi_dir.glob(f"*{str(run_id) + '*' if run_id else run_id}dwi*.nii.gz")
    )[0]
    # All DWI-related files have the same prefix name, with different file types.
    base_name = read_files.dwi.name.replace("".join(read_files.dwi.suffixes), "")
    read_files.bvals = subj_dwi_dir / (base_name + ".bval")
    read_files.bvecs = subj_dwi_dir / (base_name + ".bvec")

    # Grab the mask generated from the previous T1 freesurfer processing; this already
    # aligns with the DWIs, but will need to be resampled into the DWI space.
    read_files.mask = list(
        (oasis_processed_data_dir / f"sub-{freesurfer_id}" / "mask").glob(
            "*mask*.nii.gz"
        )
    )[0]

    bvals = torch.as_tensor(np.loadtxt(read_files.bvals).astype(int))
    bvecs = torch.as_tensor(np.loadtxt(read_files.bvecs))
    # Reshape to be N x 3
    if bvecs.shape[0] == 3:
        bvecs = bvecs.T

    dwi = torchio.ScalarImage(
        read_files.dwi,
        type=torchio.INTENSITY,
        bvals=bvals,
        bvecs=bvecs,
        reader=pitn.io.nifti_reader,
        channels_last=True,
    )

    subj_mask_dir = subj_dir / source_mask_dir
    brain_mask = torchio.LabelMap(
        read_files.mask,
        type=torchio.LABEL,
        channels_last=False,
    )
    brain_mask.set_data(brain_mask.data.bool())
    # The mask is taken from the freesurfer processing of the T1w image, which is in a
    # higher spatial resolution than the DWIs. So, need to resample the mask to the
    # DWI spacing. Typically we don't like resampling, but on the mask only should be
    # fine.
    resampler = torchio.transforms.Resample(dwi)
    brain_mask = resampler(brain_mask)
    # Slightly dilate the mask to account for registration errors between T1 and DWIs.
    np_mask = brain_mask.tensor[0].numpy().astype(bool)
    np_mask = skimage.morphology.binary_dilation(np_mask, skimage.morphology.ball(4))
    brain_mask.set_data(
        torch.from_numpy(np_mask).bool()[
            None,
        ]
    )
    
    # Create the initial Subject object to pass to the processing pipeline.
    subj_dict = torchio.Subject(subj_id=subj_id, dwi=dwi, mask=brain_mask)
    
    # Perform initial, light processing of DWIs.
    dwi_preproc = dwi_processing_transforms(subj_dict)
    # Clip extreme values in the DWIs, seen as noise.
    clipped_dwi_dict = intensity_clip_transform(
        {"dwi": dwi_preproc["dwi"].tensor * dwi_preproc["mask"].tensor}
    )
    dwi_preproc["dwi"].set_data(clipped_dwi_dict["dwi"])
    # Fit the DWIs to DTIs.
    preproc_subj = dti_fitting_transforms(dwi_preproc)

    # Save out DTIs to a file.
    header = nib.load(read_files.dwi).header
    # Both DTI and the mask files will go in the same directory, to indicate that this
    # mask is specific to the DTI space.
    subj_output_dir = oasis_processed_data_dir / f"sub-{freesurfer_id}/dti"
    subj_output_dir.mkdir(parents=True, exist_ok=True)
    write_dti_file = subj_output_dir / f"sub-{freesurfer_id}_scale-orig_dti.nii.gz"
    write_mask_file = subj_output_dir / f"sub-{freesurfer_id}_scale-orig_mask.nii.gz"

    dti_img = nib.Nifti1Image(
        preproc_subj["dti"]["data"].cpu().numpy(),
        affine=preproc_subj["dti"]["affine"],
        header=header,
    )
    mask_img = nib.Nifti1Image(
        preproc_subj["mask"]["data"].cpu().numpy(),
        affine=preproc_subj["mask"]["affine"],
        header=header,
    )

    nib.save(dti_img, write_dti_file)
    nib.save(mask_img, write_mask_file)
    # Save some of the pipeline description
    pipeline_dir = subj_output_dir / "pipeline"
    pipeline_dir.mkdir(parents=True, exist_ok=True)
    pipe_file = pipeline_dir / "description.txt"
    with open(pipe_file, "a+") as f:
        f.write("Pipeline Description\n")
        f.write(f"Voxel spacing: {params.target_vox_size}\n")
        f.write(f"Params {params.to_dict()}\n")
        f.write(
            "Pipeline functions:\n"
            + f"{(dwi_processing_transforms, intensity_clip_transform, dti_fitting_transforms)}\n"
        )
        f.write(f"dipy version {dipy.__version__}\n")
        f.write(f"nibabel version {nib.__version__}\n")
    # Optionally save into a dict for later processing.
    #     oasis_subj_data[subj_id] = preproc_subj

    print("=" * 20)

print("===Data Loaded & Transformed===")

### UVA